# DSAP 108

In [2]:
import numpy as np
import pandas as pd 
from datetime import datetime

### 讀取資料

In [4]:
grades = pd.read_csv('../../data/DSAP108 grades.csv')
v_info = pd.read_excel('../../data/DSAP108 videos_sep.xlsx', sheet_name = 'video info')
records = pd.read_excel('../../data/DSAP108 videos_sep.xlsx', sheet_name = 'watching records')
student_id = pd.read_excel('../../data/DSAP108 videos_sep.xlsx', sheet_name = 'student id')

### 觀看時間、觀看速度、平均觀看次數

In [9]:
### 觀看時間、觀看速度、平均觀看次數

# 開一個 table 存等一下計算的資料
student_info = pd.DataFrame(data = grades.drop(0)['student_id'])

# 計算總觀看時間
total_spent_time = [] # 總觀看時間
total_watch_video_time = [] # 觀看的總影片時間

for i in student_info['student_id']:
    watch_record_i = records[records['student_id'] == int(i)]
    # 抓出有在看的資料
    real_watch_i = watch_record_i[(watch_record_i['end'] > watch_record_i['start']) & (watch_record_i['playback_rate'] != 0)]
    video_time_i = real_watch_i['end'] - real_watch_i['start']# 觀看紀錄 的 影片時間(end - start)
    total_watch_video_time.append(sum(video_time_i))
    total_spent_time.append(sum(video_time_i/real_watch_i['playback_rate']))

student_info['total_watch_time'] = total_spent_time

# 計算撥放速度
total_spent_time = [x if x!= 0 else 1 for x in total_spent_time]
avg_playback_rate = [i/j for (i,j) in zip(total_watch_video_time, total_spent_time)]
student_info['avg_playback_rate'] = avg_playback_rate

# 計算影片平均觀看次數
length_l = list(v_info['meta'])# length of each lecture
'''
length_l = [( i.split('n=>')[1].split('}')[0]) for i in length_l]
length_l = [int(i) for i in length_l]
'''
total_length_l = sum(length_l)
student_info['avg_freq'] = student_info['total_watch_time']/(total_length_l/student_info['avg_playback_rate'])

student_info

,student_id,total_watch_time,avg_playback_rate,avg_freq
1,62,25957.400000,1.167336,0.405050
2,63,48334.833333,1.455989,0.940742
3,64,5044.333333,1.848807,0.124666
4,65,67902.400000,1.532936,1.391429
5,66,49396.216667,1.595203,1.053323
...,...,...,...,...
105,166,36992.416667,1.709837,0.845511
106,167,32098.033333,1.761666,0.755882
107,168,28596.833333,1.518035,0.580299
108,169,31298.883333,1.583156,0.662376


### Save data

In [10]:
student_info.to_excel('../../data/許哲/DSAP108.xlsx')